# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-01-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
f.close()


8056


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Now we need to create tables to run the following queries.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### For this table, the primary key will be (sessionId, itemInSession) since we will filter on these columns

In [12]:
create_1 = """
CREATE TABLE IF NOT EXISTS event_session_item 
(sessionId int, itemInSession int, artist text, firstName text, lastName text, song text, length float, PRIMARY KEY (sessionId, itemInSession))
"""

try:
    session.execute(create_1)
except Exception as e:
    print(e)

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_session_item (sessionId, itemInSession, artist, firstName, lastName, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[4], line[9], float(line[5])))
f.close()


The following query retrieves the songs that were heard during session ID = 338 and item in session 4.

In [14]:
query_1 = """
SELECT
    artist,
    song,
    length
FROM event_session_item
WHERE sessionId = 338
AND itemInSession = 4;
"""

try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### For this table, the primary key will be ((userId, sessionId), itemInSession) where (userId, sessionId) is the partition key and itemInSession is the clustering column since we want the results to be sorted by itemInSession.

In [15]:
create_2 = """
CREATE TABLE IF NOT EXISTS event_user_session
(userId int, sessionId int, itemInSession int, artist text, firstName text, lastName text, song text, PRIMARY KEY ((userId, sessionId), itemInSession))
"""

try:
    session.execute(create_2)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_user_session (userId, sessionId, itemInSession, artist, firstName, lastName, song) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))
        except Exception as e:
            print(e)
f.close()

In [16]:
query_2 = """
SELECT
    artist,
    song,
    firstName,
    lastName
FROM event_user_session
WHERE userId = 10
AND sessionId = 182;
"""

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### For this table, the primary key will be (song, userId) since we need to filter on song, however since there will be duplicate values if we only use song as the primary key, we add userId to make each row unique.

In [17]:
create_3 = """
CREATE TABLE IF NOT EXISTS event_song_user
(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))
"""

try:
    session.execute(create_3)
except Exception as e:
    print(e)


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_song_user (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4])
        except Exception as e:
            print(e)
f.close()

not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for format string
not enough arguments for 

In [14]:
query_3 = """
SELECT
    firstName,
    lastName
FROM event_song_user
WHERE song = 'All Hands Against His Own'
"""

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [15]:
## TO-DO: Drop the table before closing out the sessions

In [11]:
drop_query_1 = """
DROP TABLE event_session_item;
"""
drop_query_2 = """
DROP TABLE event_user_session;
"""
drop_query_3 = """
DROP TABLE event_song_user;
"""
try:
    session.execute(drop_query_1)
    session.execute(drop_query_2)
    session.execute(drop_query_3)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table event_user_session"


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()